In [1]:
import os
# import Google Drive 套件
from google.colab import drive
# 將自己的雲端硬碟掛載上去
drive.mount('/content/gdrive')

os.chdir('./gdrive/MyDrive/NLP_world/llama/llama_recipes_main/inference')      # 檔案目錄

Mounted at /content/gdrive


# 套件安裝

In [10]:
!pip install -e ./../../llama_main/.

Obtaining file:///content/gdrive/MyDrive/NLP_world/llama/llama_main
  Preparing metadata (setup.py) ... done
  Attempting uninstall: llama
    Found existing installation: llama 0.0.1
    Uninstalling llama-0.0.1:
      Successfully uninstalled llama-0.0.1
  Running setup.py develop for llama


In [11]:
!pip install transformers

In [12]:
!pip install peft

In [13]:
!pip install accelerate

In [14]:
!pip install fire

In [15]:
!pip install sentencepiece

In [16]:
!pip install fairscale

# 主程式區段2

In [9]:
# import fire
# import torch
# import os
# import sys
# path = './../../llama_main'
# if path not in sys.path:
#   sys.path.insert(1, path)

# import time
# from typing import List

# from transformers import LlamaTokenizer
# from safety_utils import get_safety_checker
# from model_utils import load_model, load_peft_model, load_llama_from_config
# from llama import Llama

# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [9]:
import json
import os
import sys
path = './../../llama_main'
if path not in sys.path:
  sys.path.insert(1, path)

import time
from pathlib import Path
from typing import List, Literal, Optional, Tuple, TypedDict
import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
from fairscale.nn.model_parallel.initialize import (
    get_model_parallel_rank,
    initialize_model_parallel,
    model_parallel_is_initialized,
)

from llama import Llama
from llama.model import ModelArgs, Transformer
from llama.tokenizer import Tokenizer

os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '29500'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'

In [17]:
ckpt_dir = './Llama-2-7b-chat'
tokenizer_path = './Llama-2-7b-chat/tokenizer.model'
max_seq_len = 1280
max_batch_size = 4

generator = Llama.build(
        ckpt_dir=ckpt_dir,
        tokenizer_path=tokenizer_path,
        max_seq_len=max_seq_len,
        max_batch_size=max_batch_size,
    )

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Loaded in 255.17 seconds


In [18]:
dialogs = [
        [
            {
                "role": "user",
                "content": "Can you summerize csv data",
            }
        ],
      ]

max_gen_len = 1280
temperature = 0.6
top_p = 0.9
results = generator.chat_completion(
    dialogs,
    max_gen_len=max_gen_len,
    temperature=temperature,
    top_p=top_p,
)

In [19]:
for dialog, result in zip(dialogs, results):
  for msg in dialog:
      print(f"{msg['role'].capitalize()}: {msg['content']}\n")
  print(
      f"> {result['generation']['role'].capitalize()}: {result['generation']['content']}"
  )
  print("\n==================================\n")

User: Can you summerize csv data

> Assistant:  Yes, you can summarize CSV data using various techniques depending on the type of summary you want to perform. Here are some common summary techniques for CSV data:
1. Averaging: Calculate the average value of a column by dividing the sum of the values in that column by the number of values.
Example: `avg_age = sum(ages) / len(ages)`
2. Counting: Count the number of unique values in a column.
Example: `count_ages = len(set(ages))`
3. Grouping: Group data in a column based on a specific criteria (e.g. by age group) and calculate the total value for each group.
Example: `group_ages = pd.groupby(ages, kind='bucket')['age_group'].sum()`
4. Ranking: Rank the values in a column based on a specific criteria (e.g. by age group).
Example: `rank_ages = pd.rank(ages, method='dense')['age_group']`
5. Summarizing with Pivot Tables: Create a pivot table to summarize data in a column.
Example: `pivot_ages = pd.pivot_table(ages, values='age', index='age_

In [28]:
data_path = './../../example_data_en.csv'

data = pd.read_csv(data_path)
data = data.iloc[:10, :]
data_markdown = data.to_markdown()


,year,industry_code_ANZSIC,industry_name_ANZSIC,rme_size_grp,variable,value,unit
0,2011,A,"Agriculture, Forestry and Fishing",a_0,Activity unit,46134,COUNT
1,2011,A,"Agriculture, Forestry and Fishing",a_0,Rolling mean employees,0,COUNT
2,2011,A,"Agriculture, Forestry and Fishing",a_0,Salaries and wages paid,279,DOLLARS(millions)
3,2011,A,"Agriculture, Forestry and Fishing",a_0,"Sales, government funding, grants and subsidies",8187,DOLLARS(millions)
4,2011,A,"Agriculture, Forestry and Fishing",a_0,Total income,8866,DOLLARS(millions)
5,2011,A,"Agriculture, Forestry and Fishing",a_0,Total expenditure,7618,DOLLARS(millions)
6,2011,A,"Agriculture, Forestry and Fishing",a_0,Operating profit before tax,770,DOLLARS(millions)
7,2011,A,"Agriculture, Forestry and Fishing",a_0,Total assets,55700,DOLLARS(millions)
8,2011,A,"Agriculture, Forestry and Fishing",a_0,Fixed tangible assets,32155,DOLLARS(millions)
9,2011,A,"Agriculture, Forestry and Fishing",b_1-5,Activity unit,21777,COUNT


In [44]:
dialogs = [
        [
            {
                "role": "system",
                "content": f"The following markdown table is about annual enterprise survey in 2021\n {data_markdown}",
            },

            {
                "role": "user",
                "content": "Please help to observe and compare the funds earned by each company",
            },
        ],
      ]

In [45]:
max_gen_len = 1280
temperature = 0.6
top_p = 0.9
results = generator.chat_completion(
    dialogs,
    max_gen_len=max_gen_len,
    temperature=temperature,
    top_p=top_p,
)

In [46]:
for dialog, result in zip(dialogs, results):
  for msg in dialog:
      print(f"{msg['role'].capitalize()}: {msg['content']}\n")
  print(
      f"> {result['generation']['role'].capitalize()}: {result['generation']['content']}"
  )
  print("\n==================================\n")

System: The following markdown table is about annual enterprise survey in 2021
 |    |   year | industry_code_ANZSIC   | industry_name_ANZSIC              | rme_size_grp   | variable                                        |   value | unit              |
|---:|-------:|:-----------------------|:----------------------------------|:---------------|:------------------------------------------------|--------:|:------------------|
|  0 |   2011 | A                      | Agriculture, Forestry and Fishing | a_0            | Activity unit                                   |   46134 | COUNT             |
|  1 |   2011 | A                      | Agriculture, Forestry and Fishing | a_0            | Rolling mean employees                          |       0 | COUNT             |
|  2 |   2011 | A                      | Agriculture, Forestry and Fishing | a_0            | Salaries and wages paid                         |     279 | DOLLARS(millions) |
|  3 |   2011 | A                      | Agricultu

# 測試用函式

In [ ]:
import torch.distributed.launch
torch.distributed.launch

<module 'torch.distributed.launch' from '/usr/local/lib/python3.10/dist-packages/torch/distributed/launch.py'>

In [ ]:
def main(
    ckpt_dir: str,
    tokenizer_path: str,
    temperature: float = 0.6,
    top_p: float = 0.9,
    max_seq_len: int = 128,
    max_gen_len: int = 64,
    max_batch_size: int = 4,
):
    generator = Llama.build(
        ckpt_dir=ckpt_dir,
        tokenizer_path=tokenizer_path,
        max_seq_len=max_seq_len,
        max_batch_size=max_batch_size,
    )

    prompts = [
        "I'm going to meet a female netizen, I hope you can bless me or give me advice",
    ]
    results = generator.text_completion(
        prompts,
        max_gen_len=max_gen_len,
        temperature=temperature,
        top_p=top_p,
    )
    for prompt, result in zip(prompts, results):
        print(prompt)
        print(f"> {result['generation']}")
        print("\n==================================\n")

In [ ]:
def main(
    ckpt_dir: str,
    tokenizer_path: str,
    temperature: float = 0.6,
    top_p: float = 0.9,
    max_seq_len: int = 512,
    max_batch_size: int = 4,
    max_gen_len = None,
):
    generator = Llama.build(
        ckpt_dir=ckpt_dir,
        tokenizer_path=tokenizer_path,
        max_seq_len=max_seq_len,
        max_batch_size=max_batch_size,
    )

    dialogs = [
        [{"role": "user", "content": "what is the recipe of mayonnaise?"}],
        [
            {"role": "user", "content": "I am going to Paris, what should I see?"},
            {
                "role": "assistant",
                "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
            },
            {"role": "user", "content": "What is so great about #1?"},
        ],
        [
            {"role": "system", "content": "Always answer with Haiku"},
            {"role": "user", "content": "I am going to Paris, what should I see?"},
        ],
        [
            {
                "role": "system",
                "content": "Always answer with emojis",
            },
            {"role": "user", "content": "How to go from Beijing to NY?"},
        ],
    ]
    results = generator.chat_completion(
        dialogs,  # type: ignore
        max_gen_len=max_gen_len,
        temperature=temperature,
        top_p=top_p,
    )

    for dialog, result in zip(dialogs, results):
        for msg in dialog:
            print(f"{msg['role'].capitalize()}: {msg['content']}\n")
        print(
            f"> {result['generation']['role'].capitalize()}: {result['generation']['content']}"
        )
        print("\n==================================\n")

In [ ]:
import os
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '5678'

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '4'

In [ ]:
generator = Llama.build(
        ckpt_dir=ckpt_dir,
        tokenizer_path=tokenizer_path,
        max_seq_len=max_seq_len,
        max_batch_size=max_batch_size,
        )

In [ ]:
## 設定參數
seed = 42
quantization = False
max_new_tokens = 100
prompt_file = "Hello llama"
do_sample = True
min_length = None
use_cache = True
top_p = 1.0
temperature= 1.0
top_k = 50
repetition_penalty = 1.0
length_penalty = 1
enable_azure_content_safety = False
enable_sensitive_topics = False
enable_saleforce_content_safety = True
use_fast_kernels = False

In [ ]:
# Set the seeds for reproducibility
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

In [ ]:
model_name = './Llama-2-7b'
model = load_model(model_name, quantization)

In [ ]:
import os
os.listdir('./')

# 下載模型

In [ ]:
#@title Llama 2 Downloader

import os


email = 'https://download.llamameta.net/*?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoiIW4%7EeFx1MDAxMD9cdTAwMDA%7EIiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZG93bmxvYWQubGxhbWFtZXRhLm5ldFwvKiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MTU5NjY1N319fV19&Signature=Ou0ts%7EwbT6TJbDs-2r1hiHJBoAW52xoi8yL1hPJH7aF-wntEqGh0a4d5Xa2RaG5GwK-BhjK-O8cof0jX1iLTGrSwC1cfZH8SCWp9pd3BLhfPcE6tkzXd8fWD-2roevlpc0SeqGPF3-0KqvfnmrHjFH6baY92Xa1HdlVCUHdhcX5I6CoyAPPnFCJVdg7-qbVXc245mGuzptlheUD6Q6cW6nF6RtdpJckscB%7EmjNA04XH17UNRDZsQyHHbkbLf-Ae5sAZIvoQYITB2kz42bT8xNMgUIF8Ej8O4wfuTz3FS4vVMSoo1uITIYN10rEeHI1cJDGuR6ECvQe51DHYOPbMUtA__&Key-Pair-Id=K15QRJLYKIFSLZ' #@param {type:"string"}
model = '7B-chat' #@param ['7B', '13B', '70B', '7B-chat', '13B-chat', '70B-chat', 'All']

if model == "All":
    model = ""


os.environ["EMAIL"] = email
os.environ["MODEL"] = model


!bash -c 'PRESIGNED_URL=$EMAIL; \
MODEL_SIZE=$MODEL; \
TARGET_FOLDER="."; \
mkdir -p ${TARGET_FOLDER}; \
echo "Downloading LICENSE and Acceptable Usage Policy"; \
wget ${PRESIGNED_URL/"*"/"LICENSE"} -O ${TARGET_FOLDER}"/LICENSE"; \
wget ${PRESIGNED_URL/"*"/"USE_POLICY.md"} -O ${TARGET_FOLDER}"/USE_POLICY.md"; \
echo "Downloading tokenizer"; \
wget ${PRESIGNED_URL/"*"/"tokenizer.model"} -O ${TARGET_FOLDER}"/tokenizer.model"; \
IFS=',' read -ra MODEL_SIZE_ARR <<< "$MODEL_SIZE"; \
for m in "${MODEL_SIZE_ARR[@]}"; \
do \
    if [[ $m == "7B" ]]; then \
        SHARD=0; \
        MODEL_PATH="llama-2-7b"; \
    elif [[ $m == "7B-chat" ]]; then \
        SHARD=0; \
        MODEL_PATH="llama-2-7b-chat"; \
    elif [[ $m == "13B" ]]; then \
        SHARD=1; \
        MODEL_PATH="llama-2-13b"; \
    elif [[ $m == "13B-chat" ]]; then \
        SHARD=1; \
        MODEL_PATH="llama-2-13b-chat"; \
    elif [[ $m == "70B" ]]; then \
        SHARD=7; \
        MODEL_PATH="llama-2-70b"; \
    elif [[ $m == "70B-chat" ]]; then \
        SHARD=7; \
        MODEL_PATH="llama-2-70b-chat"; \
    fi; \
    echo "Downloading $MODEL_PATH"; \
    mkdir -p ${TARGET_FOLDER}/$MODEL_PATH; \
    for s in $(seq -f "0%g" 0 $SHARD); \
    do \
        wget ${PRESIGNED_URL/"*"/"${MODEL_PATH}/consolidated.${s}.pth"} -O ${TARGET_FOLDER}/$MODEL_PATH/consolidated.$s.pth; \
    done; \
    wget ${PRESIGNED_URL/"*"/"${MODEL_PATH}/params.json"} -O ${TARGET_FOLDER}/$MODEL_PATH/params.json; \
done'
